In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import tensorflow as tf
import networkx as nx
import random
import pickle
import itertools
import math
import urllib.request as urllib
import io
import zipfile

from sklearn.metrics import accuracy_score, precision_score, recall_score
from sklearn.model_selection import train_test_split
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras import layers, losses
from tensorflow.keras.models import Model
from tensorflow.linalg import diag
from tensorflow.keras import callbacks
from sklearn.cluster import KMeans
from sklearn.metrics.cluster import normalized_mutual_info_score
from networkx.generators.community import LFR_benchmark_graph
from itertools import count
from scipy.spatial import distance_matrix

from helpers import *
from Autoencoder import *

In [ ]:
#POLBOOKS
polbooks = nx.read_gml("graphs/polbooks.gml")

In [ ]:
A = create_adjacency_matrix(polbooks)

#modularity
#X_polbooks_1 = tf.convert_to_tensor(nx.modularity_matrix(polbooks).astype('float32'))

#similarity
X_polbooks_1 = tf.convert_to_tensor(adjacency_to_similarity(A).astype('float32'))

In [ ]:
#Stacked autoencoder model
ae_polbooks_1 = Autoencoder(105, 64, k_reg=tf.keras.regularizers.L2(), act_reg=SparseRegularizer())
history_polbooks_1 = train(ae_polbooks_1, 5000, 105, X_polbooks_1)

X_polbooks_2 = ae_polbooks_1.encoder(X_polbooks_1)
ae_polbooks_2 = Autoencoder(64, 32, k_reg=tf.keras.regularizers.L2(), act_reg=SparseRegularizer())
history_polbooks_2 = train(ae_polbooks_2, 5000, 105, X_polbooks_2)

X_polbooks_3 = ae_polbooks_2.encoder(X_polbooks_2)
ae_polbooks_3 = Autoencoder(32, 16, k_reg=tf.keras.regularizers.L2(), act_reg=SparseRegularizer())
history_polbooks_3 = train(ae_polbooks_3, 5000, 105, X_polbooks_3)

In [ ]:
H_polbooks = ae_polbooks_3.encoder(X_polbooks_3)
labels = polbooks_labels(polbooks)
kmeans_polbooks = KMeans(n_clusters=3, n_init=20).fit(H_polbooks)
normalized_mutual_info_score(labels, kmeans_polbooks.labels_)

In [ ]:
combined_encoder = tf.keras.Sequential()
combined_decoder = tf.keras.Sequential()

combined_encoder.add(ae_polbooks_1.encoder.layers[0])
combined_encoder.add(ae_polbooks_2.encoder.layers[0])
combined_encoder.add(ae_polbooks_3.encoder.layers[0])

combined_decoder.add(ae_polbooks_3.decoder.layers[0])
combined_decoder.add(ae_polbooks_2.decoder.layers[0])
combined_decoder.add(ae_polbooks_1.decoder.layers[0])

#fine tuning
ae_combined=Autoencoder(105, 16, learning_rate=0.0001)
ae_combined.encoder=combined_encoder
ae_combined.decoder=combined_decoder

ae_combined.build(input_shape=(None, 105))

combined_history = train(ae_combined, 500, 2708, X_polbooks_1)

In [ ]:
H_polbooks = ae_combined.encoder(X_polbooks_1)
labels = polbooks_labels(polbooks)
kmeans_polbooks = KMeans(n_clusters=3, random_state=0).fit(H_polbooks)
normalized_mutual_info_score(labels, kmeans_polbooks.labels_)